Implémentation de résolution numérique approchée des 2 équations couplées du mouvement de précession :
$\frac{dv_x}{dt} = \omega v_y(t), 
\frac{dv_y}{dt} = -\omega v_x(t), 
v_x(0) = V_0,
v_y(0) = 0$
à l'aide de la la méthode de descente de gradients sur la fonction d'erreur


In [58]:
#imports
from numpy import pi, cos, sin, dot, zeros, linspace, arange, matmul
from numpy.random import randn

Paramètres du modèle

In [59]:
#nombre de coefficients de Fourier ajustables
M = 10
M_range = arange(M) +1

# nombre de points pour la variable indépendante
N = 100
T = linspace(0,1,N)

Paramètres du problème

In [60]:
W = 2*pi #pulsation
V0 = 1 #vitesse initiale

Calculs des vecteurs des cosinus et sinus qui seront souvent utilisés

In [61]:
#matrice de coefficients (m*W*T[i]) de taille NxM
#avec première coordonnée i, et deuxème m
mat = W*matmul(T.reshape((N,1)), M_range.reshape((1,M)))
cos = cos(mat)
sin = sin(mat)

Définition du gradient

In [62]:
def calcGrad(A,B) :
    #calcule le gradient de l'erreur par rapport aux
    #2 vecteurs de paramètres A et B
    grad_A = zeros((M))
    grad_B = zeros((M))

    for i in range(N) :
        #valeurs de vx et vy à l'instant T[i]
        vx = V0 + dot(A, cos[i]-1) + dot(B, sin[i])
        vy = - dot(A, sin[i]) + dot(B, cos[i]-1)

        #valeurs des dérivées de vx et vy à l'instant T[i]
        dvx = W*(-dot(M_range*A, sin[i]) + dot(M_range*B, cos[i]))
        dvy = W*(-dot(M_range*A, cos[i]) - dot(M_range*B, sin[i]))

        #différences
        ex = dvx - W*vy
        ey = dvy + W*vx

        #on incrémente les gradients
        #pour A
        dex_dA = W*(1-M_range)*sin[i]
        dey_dA = W*((1-M_range)*cos[i]-1)
        grad_A += ex*dex_dA + ey*dey_dA
        #pour B
        dex_dB = W*((M_range-1)*cos[i]+1)
        dey_dB = W*(1-M_range)*sin[i]
        grad_B += ex*dex_dB + ey*dey_dB

    return grad_A, grad_B

Paramètres d'apprentissage

In [63]:
alpha = 1e-6 #taux d'apprentissage pour la descente de gradients
epochs = 10000 #nombre d'itération

#initialisation des coefficients
A = randn((M))
B = randn((M))

Entraînement

In [64]:
for k in range(epochs) :
    if k%(epochs//10)==0 :
        print("\nCoefficients après",k,"itérations :\n",A,"\n", B)

    grad_A, grad_B = calcGrad(A,B)
    A -= alpha*grad_A
    B -= alpha*grad_B


Coefficients après 0 itérations :
 [ 1.95000439 -0.77398224 -0.5615855   0.6122229  -0.67366247 -0.38463424
 -0.18402382  0.80691276 -1.51701531 -0.50711993] 
 [-0.32217747  0.35345041 -1.53602392  0.33126212 -0.07307273 -1.22874455
 -0.27373392  0.78600582  0.58344601 -1.84052923]

Coefficients après 1000 itérations :
 [ 1.30157622e+00 -1.46620209e-01 -2.60626776e-02 -1.07807631e-02
 -5.82524288e-03 -3.61145789e-03 -2.43645342e-03 -1.74074793e-03
 -1.29631331e-03 -9.96038534e-04] 
 [-0.07773726  0.03835405  0.00656636  0.00273473  0.00147945  0.00091744
  0.00061891  0.00044209  0.00032911  0.00025279]

Coefficients après 2000 itérations :
 [ 1.08237767e+00 -4.00213824e-02 -7.12705853e-03 -2.94713020e-03
 -1.59235110e-03 -9.87192909e-04 -6.66007315e-04 -4.75840453e-04
 -3.54357714e-04 -2.72280021e-04] 
 [-2.12905612e-02  1.03440144e-02  1.84186711e-03  7.61651038e-04
  4.11525827e-04  2.55129466e-04  1.72122444e-04  1.22975779e-04
  9.15798051e-05  7.03676620e-05]

Coefficients après

Affichage des résultats

In [65]:
print()
print("Coefficients finaux après",epochs,"itérations :",A,B)


Coefficients finaux après 10000 itérations : [ 1.00000255e+00 -1.23919994e-06 -2.20679520e-07 -9.12537244e-08
 -4.93048945e-08 -3.05670278e-08 -2.06219718e-08 -1.47337251e-08
 -1.09721848e-08 -8.43076573e-09] [-6.59235880e-07  3.20274560e-07  5.70352161e-08  2.35847707e-08
  1.27429827e-08  7.90013061e-09  5.32980412e-09  3.80797092e-09
  2.83579070e-09  2.17895410e-09]
